In [3]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate, prompt
from langchain.chains import LLMChain

csv_paths = {
    "Patient": "/Users/sanket/Documents/Python Projects/Windsurf_Tutorial/csv_files/Patient.csv",
    "Observation": "/Users/sanket/Documents/Python Projects/Windsurf_Tutorial/csv_files/Observation.csv",
    "MedicationRequest": "/Users/sanket/Documents/Python Projects/Windsurf_Tutorial/csv_files/MedicationRequest.csv",
    "DocumentReference": "/Users/sanket/Documents/Python Projects/Windsurf_Tutorial/csv_files/DocumentReference.csv",
    "Condition": "/Users/sanket/Documents/Python Projects/Windsurf_Tutorial/csv_files/Condition.csv",
    "Encounter": "/Users/sanket/Documents/Python Projects/Windsurf_Tutorial/csv_files/Encounter.csv"
}

dfs = {name: pd.read_csv(value) for name, value in csv_paths.items()}

# Function to build GPT Friendly SUmmary

def generate_patient_summary(patient_id):
    summary = f"paitient summary for {patient_id}:\n\n"
    for section, df in dfs.items():
        records = df[df['PatientID'] == patient_id]
        if not records.empty:
            summary += f"\n{section}:\n"
            for _, row in records.iterrows():
                for col, val in row.items():
                    summary += f"{col}: {val}\n"
                summary += "\n"
    return summary


In [4]:
# Test the function
patient_id = "patient-0001"
summary = generate_patient_summary(patient_id)

# Print the first 1000 characters to keep output manageable
print(summary[:1000] + ("..." if len(summary) > 1000 else ""))

# Print total length to give an idea of the full summary size
print(f"\nTotal summary length: {len(summary)} characters")

paitient summary for patient-0001:


Patient:
PatientID: patient-0001
Name: Mark Johnson
Gender: male
BirthDate: 1971-11-24


Observation:
PatientID: patient-0001
ObservationDate: 2025-03-17T23:46:53.561861
Category: vital-signs
Code: Blood Pressure
Systolic: 131
Diastolic: 87


MedicationRequest:
PatientID: patient-0001
MedicationDate: 2025-03-17T23:46:53.561861
Medication: Lisinopril
Dosage: Take one tablet daily


DocumentReference:
PatientID: patient-0001
NoteDate: 2025-06-07T23:46:53.561861
NoteText: SOAP Note:
Subjective: Mark reports fatigue.
Objective: BP 133/78 mmHg, HR 75 bpm.
Assessment: Hypertension.
Plan: Prescribe medication.


Condition:
PatientID: patient-0001
ConditionCode: Hypertension
ClinicalStatus: active
OnsetDate: 2019-06-03T23:46:53.561861


Encounter:
PatientID: patient-0001
EncounterID: encounter-0001
Start: 2025-03-17T23:46:53.561861
End: 2025-06-07T23:46:53.561861
Status: finished
Class: AMB



Total summary length: 934 characters


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

# Load environment variables (make sure you have OPENAI_API_KEY in your .env file)
load_dotenv()

def setup_llm_chain():
    # Initialize the language model
    llm = ChatOpenAI(
        model_name="gpt-4",  # or "gpt-3.5-turbo" for faster/cheaper responses
        temperature=0.7,
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Define the prompt template
    prompt_template = """
    You are a helpful medical assistant analyzing patient data. 
    Given the following patient summary, answer the user's question.
    If you don't know the answer, say you don't know, don't try to make up an answer.
    
    Patient Summary:
    {patient_summary}
    
    Question: {question}
    
    Answer:"""
    
    prompt = ChatPromptTemplate.from_template(prompt_template)
    
    # Create the chain
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    return llm_chain

def ask_about_patient(patient_summary, question, llm_chain):
    """
    Ask a question about a patient's data using the LLM chain.
    
    Args:
        patient_summary (str): The generated patient summary
        question (str): The question to ask about the patient
        llm_chain: The initialized LLM chain
        
    Returns:
        str: The model's response
    """
    response = llm_chain.run(patient_summary=patient_summary, question=question)
    return response

# Example usage:
if __name__ == "__main__":
    # Initialize the chain
    llm_chain = setup_llm_chain()
    
    # Generate a patient summary (using your existing function)
    patient_id = "patient-0001"
    summary = generate_patient_summary(patient_id)
    
    # Example questions
    questions = [
        "What are the patient's current medications?",
        "What conditions has this patient been diagnosed with?",
        "When was the patient's last encounter?",
        "Summarize this patient's medical history in 3-5 bullet points."
    ]
    
    # Ask each question and print responses
    for question in questions:
        print(f"\nQuestion: {question}")
        answer = ask_about_patient(summary, question, llm_chain)
        print(f"Answer: {answer}\n{'-'*50}")

/var/folders/jx/d374wy8j1mdfpkk2hksh8q_w0000gn/T/ipykernel_61530/4269704639.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/jx/d374wy8j1mdfpkk2hksh8q_w0000gn/T/ipykernel_61530/4269704639.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/var/folders/jx/d374wy8j1mdfpkk2hksh8q_w0000gn/T/ipykernel_61530/4269704639.py:50: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_cha


Question: What are the patient's current medications?
Answer: The patient's current medication is Lisinopril with a dosage of one tablet daily.
--------------------------------------------------

Question: What conditions has this patient been diagnosed with?
Answer: The patient has been diagnosed with Hypertension.
--------------------------------------------------

Question: When was the patient's last encounter?
Answer: The patient's last encounter was on 2025-06-07T23:46:53.561861.
--------------------------------------------------

Question: Summarize this patient's medical history in 3-5 bullet points.
Answer: - Mark Johnson, a male patient born on November 24, 1971, has been diagnosed with active Hypertension since June 3, 2019.
- His blood pressure was recorded as 131/87 mmHg on March 17, 2025, and 133/78 mmHg on June 7, 2025.
- He was prescribed Lisinopril, with a dosage of one tablet daily on March 17, 2025, as a part of his treatment plan.
- He had an encounter with a healt

In [9]:
df = pd.read_csv('/Users/sanket/Documents/Python Projects/Windsurf_Tutorial/csv_files/ProviderPatientDashboard.csv')
df.head()


/Users/sanket/Library/Python/3.9/lib/python/site-packages/IPython/core/formatters.py:344: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/Users/sanket/Library/Python/3.9/lib/python/site-packages/IPython/core/formatters.py:344: FutureWarning: RangeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,PatientID,ProviderID,VisitDate,Status_x,ProviderName,Name,Gender,BirthDate,ObservationDate,Category,...,ClinicalStatus,OnsetDate,EncounterID,Start,End,Status_y,Class,CombinedText,ClinicalFocusText,ClinicalFocusEmbedding
0,patient-0001,P004,2025-06-21,Pending,Dr. Ahmed Khan,Mark Johnson,male,1971-11-24,2025-03-17T23:46:53.561861,vital-signs,...,active,2019-06-03T23:46:53.561861,encounter-0001,2025-03-17T23:46:53.561861,2025-06-07T23:46:53.561861,finished,AMB,PatientID: patient-0001 | Name: Mark Johnson |...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.0699615404009819, -0.15284700691699982, 0...."
1,patient-0001,P001,2025-08-28,Completed,Dr. Alice Smith,Mark Johnson,male,1971-11-24,2025-03-17T23:46:53.561861,vital-signs,...,active,2019-06-03T23:46:53.561861,encounter-0001,2025-03-17T23:46:53.561861,2025-06-07T23:46:53.561861,finished,AMB,PatientID: patient-0001 | Name: Mark Johnson |...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.0699615404009819, -0.15284700691699982, 0...."
2,patient-0002,P001,2025-06-01,Cancelled,Dr. Alice Smith,David Taylor,male,1948-06-01,2025-06-16T23:46:53.562317,vital-signs,...,active,2019-09-04T23:46:53.562317,encounter-0002,2025-06-16T23:46:53.562317,2025-04-10T23:46:53.562317,finished,AMB,PatientID: patient-0002 | Name: David Taylor |...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.024824656546115875, -0.10685153305530548, ..."
3,patient-0002,P002,2025-06-12,Pending,Dr. John Patel,David Taylor,male,1948-06-01,2025-06-16T23:46:53.562317,vital-signs,...,active,2019-09-04T23:46:53.562317,encounter-0002,2025-06-16T23:46:53.562317,2025-04-10T23:46:53.562317,finished,AMB,PatientID: patient-0002 | Name: David Taylor |...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.024824656546115875, -0.10685153305530548, ..."
4,patient-0003,P002,2025-06-24,No-show,Dr. John Patel,Michael Mcclain,male,1946-04-19,2025-04-26T23:46:53.562601,vital-signs,...,active,2017-05-13T23:46:53.562601,encounter-0003,2025-04-26T23:46:53.562601,2025-05-08T23:46:53.562601,finished,AMB,PatientID: patient-0003 | Name: Michael Mcclai...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.02864755317568779, -0.09952106326818466, 0..."


In [2]:
import pandas as pd

df = pd.read_csv("/Users/sanket/Documents/Python Projects/Windsurf_Tutorial copy 7/csv_files/ProviderPatientDashboard.csv")
df.head()


,PatientID,ProviderID,VisitDate,Status_x,ProviderName,Name,Gender,BirthDate,ObservationDate,Category,...,ClinicalStatus,OnsetDate,EncounterID,Start,End,Status_y,Class,CombinedText,ClinicalFocusText,ClinicalFocusEmbedding
0,patient-0001,P004,2025-06-21,Pending,Dr. Ahmed Khan,Mark Johnson,male,1971-11-24,2025-03-17T23:46:53.561861,vital-signs,...,active,2019-06-03T23:46:53.561861,encounter-0001,2025-03-17T23:46:53.561861,2025-06-07T23:46:53.561861,finished,AMB,PatientID: patient-0001 | Name: Mark Johnson |...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.0699615404009819, -0.15284700691699982, 0...."
1,patient-0001,P001,2025-08-28,Completed,Dr. Alice Smith,Mark Johnson,male,1971-11-24,2025-03-17T23:46:53.561861,vital-signs,...,active,2019-06-03T23:46:53.561861,encounter-0001,2025-03-17T23:46:53.561861,2025-06-07T23:46:53.561861,finished,AMB,PatientID: patient-0001 | Name: Mark Johnson |...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.0699615404009819, -0.15284700691699982, 0...."
2,patient-0002,P001,2025-06-01,Cancelled,Dr. Alice Smith,David Taylor,male,1948-06-01,2025-06-16T23:46:53.562317,vital-signs,...,active,2019-09-04T23:46:53.562317,encounter-0002,2025-06-16T23:46:53.562317,2025-04-10T23:46:53.562317,finished,AMB,PatientID: patient-0002 | Name: David Taylor |...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.024824656546115875, -0.10685153305530548, ..."
3,patient-0002,P002,2025-06-12,Pending,Dr. John Patel,David Taylor,male,1948-06-01,2025-06-16T23:46:53.562317,vital-signs,...,active,2019-09-04T23:46:53.562317,encounter-0002,2025-06-16T23:46:53.562317,2025-04-10T23:46:53.562317,finished,AMB,PatientID: patient-0002 | Name: David Taylor |...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.024824656546115875, -0.10685153305530548, ..."
4,patient-0003,P002,2025-06-24,No-show,Dr. John Patel,Michael Mcclain,male,1946-04-19,2025-04-26T23:46:53.562601,vital-signs,...,active,2017-05-13T23:46:53.562601,encounter-0003,2025-04-26T23:46:53.562601,2025-05-08T23:46:53.562601,finished,AMB,PatientID: patient-0003 | Name: Michael Mcclai...,Medication: Lisinopril | NoteText: SOAP Note:\...,"[-0.02864755317568779, -0.09952106326818466, 0..."


In [ ]:
print(df["ClinicalFocusText"][0])
print(df["ClinicalFocusEmbedding"][0])

Medication: Lisinopril | NoteText: SOAP Note:
Subjective: Mark reports fatigue.
Objective: BP 133/78 mmHg, HR 75 bpm.
Assessment: Hypertension.
Plan: Prescribe medication. | ConditionCode: Hypertension
[-0.0699615404009819, -0.15284700691699982, 0.03447654843330383, 0.060571763664484024, 0.021952956914901733, -0.2816104590892792, 0.19421836733818054, -0.0439448244869709, 0.1923183649778366, 0.293134868144989, -0.08811387419700623, 0.2264627367258072, -0.15877968072891235, 0.0028016038704663515, -0.06869811564683914, -0.20605425536632538, -0.00638000201433897, -0.14153321087360382, -0.1407172977924347, -0.05722324550151825, -0.11512982845306396, -0.21412256360054016, -0.2999666929244995, 0.014468804933130741, -0.13021451234817505, -0.3178921937942505, 0.12998461723327637, 0.7696101665496826, 0.06546011567115784, 0.41862931847572327, 0.06510976701974869, 0.24565745890140533, -0.20244108140468597, -0.0534510612487793, -0.11792107671499252, 0.10506808012723923, -0.002444232115522027, 0.028